In [1]:
import pandas as pd
import requests
import time 
import random

In [2]:
url_saldos_totales = "https://riskzone.anywhereportfolio.com.ar:9099/api/saldosconsolidados"
tokenobj = {'key': 'value'}
REQUEST_GENERAL = requests.get(url_saldos_totales, json = tokenobj)
data  = (REQUEST_GENERAL.json())
df = pd.DataFrame(data)

### Dejar solo finalidad margenes

In [6]:
df = df[df["FinalidadID"] == 2]


,MiembroCompensadorID,CuentaCompensacionID,FinalidadID,MonedaID,MonedaDescripcion,Cotizacion,MiembroCompensadorCodigo,CuentaCompensacionCodigo,FinalidadDescripcion,EgresoNoVerificado,...,SaldoInicialMoneda,EgresoVerificadoMoneda,IngresoVerificadoMoneda,CuitCuil,CuentaNeteoDescripcion,MargenRequeridoAnterior,MargenRequeridoDelDia,MargenRequeridoTotal,MargenRequeridoAnteriorSC,HasCuentaInfoEmpty
0,694,1001,2,2,Pesos,1.00,320,1001,Márgenes,0.0,...,0.000000e+00,0.0,0.0,30541906564,BARRILLI SOCIEDAD ANONIMA,-2.857369e+06,0.000000,-2.857369e+06,-2.857369e+06,False
1,694,1001,2,2,Pesos,1.00,320,1001,Márgenes,0.0,...,0.000000e+00,0.0,0.0,30708237996,AG CAMPOALTO SA,5.488949e+05,0.000000,5.488949e+05,0.000000e+00,False
2,694,1001,2,2,Pesos,1.00,320,1001,Márgenes,0.0,...,0.000000e+00,0.0,0.0,30673999308,OSCAR ROBERTO DI SCIASCIO SRL,-1.526100e+05,0.000000,-1.526100e+05,-1.526100e+05,False
3,694,1001,2,2,Pesos,1.00,320,1001,Márgenes,0.0,...,0.000000e+00,0.0,0.0,30711169748,SERVIGRAIN S.A.,-3.878486e+07,5385.169780,-3.877948e+07,-3.878486e+07,False
5,694,1001,2,2,Pesos,1.00,320,1001,Márgenes,0.0,...,0.000000e+00,0.0,0.0,27249263201,BOLATTI IVANA NATALIA,6.178184e+04,-1938.125338,5.984372e+04,0.000000e+00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9951,22,2022,2,1,Dólar MtR,152.61,22,2022,Márgenes,0.0,...,2.446500e+06,0.0,0.0,30539435589,GRASSI S.A.,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,False
9952,35,4140,2,1,Dólar MtR,152.61,35,4140,Márgenes,0.0,...,3.434400e+04,0.0,0.0,30663290262,INTAGRO SOCIEDAD ANONIMA,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,False
9956,256,6147,2,5,Dólar MEP,308.51,256,6147,Márgenes,0.0,...,1.515000e+06,0.0,0.0,30500010912,BANCO DE LA NACION ARGENTINA,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,False
9957,285,7267,2,5,Dólar MEP,308.51,285,7267,Márgenes,0.0,...,1.174072e+05,0.0,0.0,30680794622,MERRILL LYNCH ARGENTINA SOCIEDAD ANONIMA,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,False


In [18]:
ejemplo = df[df["CuentaNeteoID"] == 40741]
ejemplo = ejemplo.groupby("CuentaNeteoCodigo")["SaldoInicial"].sum()
ejemplo.item()

float